# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.core.environment import Environment 
from azureml.core.model import InferenceConfig 
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model

import collections
# Due to the not working widgets library at Python 3.10+
if not hasattr(collections, 'Mapping'):
    import collections.abc
    collections.Mapping = collections.abc.Mapping
    collections.Sequence = collections.abc.Sequence
    collections.Iterable = collections.abc.Iterable
    collections.Callable = collections.abc.Callable

from azureml.widgets import RunDetails

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# choose a name for experiment
experiment_name = 'AutoML-HFP-Exp'
project_folder = './automl'

experiment=Experiment(ws, experiment_name)
experiment

quick-starts-ws-293616
aml-quickstarts-293616
westus2
510b94ba-e453-4417-988b-fbdc37b55ca7


Name,Workspace,Report Page,Docs Page
AutoML-HFP-Exp,quick-starts-ws-293616,Link to Azure Machine Learning studio,Link to Documentation


In [3]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
found = False
key = "heart_failure_clinical_records"
description_text = ""

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://github.com/onuroezcelik/nd00333_Capstone_Project_Machine_Learning_Engineer/blob/master/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

print("The registered dataset is found: ",found)
df = dataset.to_pandas_dataframe()
df.describe()

The registered dataset is found:  True
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [4]:
dataset.take(5).to_pandas_dataframe()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## AutoML Configuration

Explain why you chose the automl settings and cofiguration you used below.

In [5]:
# Choose a name for your CPU cluster
amlcompute_cluster_name = "automlcluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster.')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Creating a new compute cluster.
InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded.....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [9]:
# automl settings
automl_settings = {
    "experiment_timeout_minutes": 15,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# automl config
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [10]:
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
AutoML-HFP-Exp,AutoML_77193095-7f95-4209-955e-b75c2e2b2485,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

2026-01-13 08:51:18.604418: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.


A Jupyter Widget

Experiment,Id,Type,Status,Details Page,Docs Page
AutoML-HFP-Exp,AutoML_77193095-7f95-4209-955e-b75c2e2b2485,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  In order to accurately evaluate the model(s) trained by AutoML, we leverage a dataset that the model is not trained on. Hence, if the user doesn't provide an explicit validation dataset, a part of the training dataset is used to achieve this. For smaller datasets (fewer than 20,000 samples), cross-validation is leveraged, else a single hold-out set is split from the training data to serve as the validation dataset. Hence, for your input data we leverage cross-validation with 10 folds, if the number of training samples are fewer than 1000, and 3 folds in all other cases.
              Learn mo

{'runId': 'AutoML_77193095-7f95-4209-955e-b75c2e2b2485',
 'target': 'automlcluster',
 'status': 'Completed',
 'startTimeUtc': '2026-01-13T08:51:28.85934Z',
 'endTimeUtc': '2026-01-13T09:14:23.575043Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 15 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'automlcluster',
  'AMLSettingsJsonString': '{"path":null,"name":"AutoML-HFP-Exp","subscription_id":"510b94ba-e453-4417-988b-fbdc37b55ca7","resource_group":"aml-quickstarts-293616","workspace_name":"quick-starts-ws-293616","region":"westus2","compute_target":"automlcluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"en

## Best Model

In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [15]:
results = []

for child in remote_run.get_children():
    name = child.properties.get('run_algorithm')
    metrics = child.get_metrics()
    acc = metrics.get('accuracy', None)

    if acc is not None:
        results.append((name, acc))

results_sorted = sorted(results, key=lambda x: x[1], reverse=True)

for name, acc in results_sorted:
    print(f"{name}: {acc}")

VotingEnsemble: 0.8760919540229886
StackEnsemble: 0.859425287356322
XGBoostClassifier: 0.859425287356322
XGBoostClassifier: 0.8526436781609196
LightGBM: 0.8526436781609196
ExtremeRandomTrees: 0.8458620689655174
RandomForest: 0.8427586206896553
LightGBM: 0.8360919540229885
XGBoostClassifier: 0.835977011494253
LightGBM: 0.8359770114942527
XGBoostClassifier: 0.8358620689655174
XGBoostClassifier: 0.8328735632183909
RandomForest: 0.8327586206896551
RandomForest: 0.8325287356321839
XGBoostClassifier: 0.8294252873563218
RandomForest: 0.8290804597701149
XGBoostClassifier: 0.825977011494253
LogisticRegression: 0.8258620689655173
RandomForest: 0.8226436781609194
XGBoostClassifier: 0.8225287356321838
LightGBM: 0.8193103448275864
LogisticRegression: 0.8159770114942528
XGBoostClassifier: 0.8126436781609195
XGBoostClassifier: 0.8093103448275862
LogisticRegression: 0.792528735632184
RandomForest: 0.7889655172413793
KNN: 0.7658620689655172
LogisticRegression: 0.742183908045977
SVM: 0.7357471264367816


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [13]:
# Save the best model
best_automl_run, best_automl_model= remote_run.get_output()
best_automl_model = best_automl_run.register_model(model_name="best_automl_model", model_path="outputs/")

Package:azureml-automl-runtime, training version:1.61.0, current version:1.60.0
Package:azureml-core, training version:1.61.0.post1, current version:1.60.0
Package:azureml-dataprep, training version:5.4.2, current version:5.1.6
Package:azureml-dataprep-native, training version:42.1.0, current version:41.0.0
Package:azureml-dataprep-rslex, training version:2.25.2, current version:2.22.5
Package:azureml-dataset-runtime, training version:1.61.0, current version:1.60.0
Package:azureml-defaults, training version:1.61.0, current version:1.60.0
Package:azureml-inference-server-http, training version:1.5.0, current version:1.4.0
Package:azureml-interpret, training version:1.61.0, current version:1.60.0
Package:azureml-mlflow, training version:1.61.0.post1, current version:1.60.0
Package:azureml-pipeline-core, training version:1.61.0, current version:1.60.0
Package:azureml-responsibleai, training version:1.61.0, current version:1.60.0
Package:azureml-telemetry, training version:1.61.0, current 

In the cell below, send a request to the web service you deployed to test it.

In [17]:
environment = best_automl_run.get_environment()
entry_script='scoring.py'
best_automl_run.download_file('outputs/scoring_file_v_1_0_0.py', entry_script)


inference_config = InferenceConfig(entry_script = entry_script, environment = environment)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                    memory_gb = 1, 
                                                    auth_enabled= True, 
                                                    enable_app_insights= True)

service = Model.deploy(ws, "bestmodeldeployv1", [best_automl_model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2026-01-13 11:40:58+00:00 Registering the environment.
2026-01-13 11:40:59+00:00 Generating deployment configuration.
2026-01-13 11:41:07+00:00 Submitting deployment to compute.
Failed


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Transitioning
Operation ID: b860f094-e846-4337-a34a-264e32c682e2
More information can be found using '.get_logs()'
Error:
{
  "code": "AuthorizationFailed",
  "statusCode": 403,
  "message": "ACI Service request failed. Reason: The client 'b6a80bde-9bf4-4b65-83d1-42cc5618b845' with object id 'c231fbe4-980d-4246-8d28-5321c88a8a8c' does not have authorization to perform action 'Microsoft.ContainerInstance/containerGroups/write' over scope '/subscriptions/510b94ba-e453-4417-988b-fbdc37b55ca7/resourceGroups/aml-quickstarts-293616/providers/Microsoft.ContainerInstance/containerGroups/bestmodeldeployv1-1i_TK8ME4ES7AVsrIbeQdg' or the scope is invalid. If access was recently granted, please refresh your credentials.."
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Transitioning
Operation ID: b860f094-e846-4337-a34a-264e32c682e2
More information can be found using '.get_logs()'
Error:
{
  "code": "AuthorizationFailed",
  "statusCode": 403,
  "message": "ACI Service request failed. Reason: The client 'b6a80bde-9bf4-4b65-83d1-42cc5618b845' with object id 'c231fbe4-980d-4246-8d28-5321c88a8a8c' does not have authorization to perform action 'Microsoft.ContainerInstance/containerGroups/write' over scope '/subscriptions/510b94ba-e453-4417-988b-fbdc37b55ca7/resourceGroups/aml-quickstarts-293616/providers/Microsoft.ContainerInstance/containerGroups/bestmodeldeployv1-1i_TK8ME4ES7AVsrIbeQdg' or the scope is invalid. If access was recently granted, please refresh your credentials.."
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal stat

In [ ]:
%run endpoint.py

In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

In [ ]:
best_automl_run.download_file('outputs/conda_env_v_1_0_0.yml', 'env.yml')

In [ ]:
service.delete()

In [ ]:
compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
